In [1]:
import sys
import boto3
import pandas as pd
from datetime import datetime

In [ ]:
def main():
    try:
        dynamodb = boto3.resource('dynamodb',
                             region_name='ap-northeast-2',
                             endpoint_url='http://dynamodb.ap-northeast-2.amazonaws.com')
    except:
        print('DynamoDB접속오류!')
        sys.exit(1)
    table = dynamodb.Table('top_tracks')
    response = table.scan()
    response = response['Items']
    data = pd.DataFrame(response)
    data = data[['artist_id','track_id','track_name','popularity','external_url']]
    data.to_parquet('top_tracks.parquet', engine='pyarrow', compression='snappy')
    dt = datetime.utcnow().strftime('%Y-%m-%d')
    s3 = boto3.resource('s3', region_name='ap-northeast-2')
    ob = s3.Object('moon-artist', 'top-tracks/dt={}/top_tracks.parquet'.format(dt))
    data = open('top_tracks.parquet', 'rb')
    ob.put(Body=data)


In [ ]:
if __name__ == '__main__':
    main()